## Notebook Imports

In [1]:
import pandas as pd
import numpy as np

## Constants

In [64]:
TRAINING_DATA_FILE = 'SpamData/02_Training/train-data.txt'
TESTING_DATA_FILE = 'SpamData/02_Training/test-data.txt'

TOKEN_SPAM_PROB_FILE = 'SpamData/03_Testing/prob-spam.txt'
TOKEN_HAM_PROB_FILE = 'SpamData/03_Testing/prob-ham.txt'
TOKEN_ALL_PROB_FILE = 'SpamData/03_Testing/prob-all-tokens.txt'

TEST_FEATURE_MATRIX = 'SpamData/03_Testing/test-features.txt'
TEST_TARGET_FILE = 'SpamData/03_Testing/test-target.txt'

VOCAB_SIZE = 2500

## Read and Load Features from .txt Files into Numpy Array

In [3]:
sparse_train_data = np.loadtxt(TRAINING_DATA_FILE, delimiter=' ', dtype=int)

In [4]:
sparse_test_data = np.loadtxt(TESTING_DATA_FILE, delimiter=' ', dtype=int)

In [73]:
sparse_train_data[-5:]

array([[5798, 2061,    0,    2],
       [5798, 2062,    0,    1],
       [5798, 2291,    0,    1],
       [5798, 2364,    0,    1],
       [5798, 2417,    0,    1]])

In [6]:
print('No of rows in training file:', sparse_train_data.shape[0])
print('No of rows in testing file:', sparse_test_data.shape[0])

No of rows in training file: 263494
No of rows in testing file: 112511


In [7]:
print('No of emails in training file:', np.unique(sparse_train_data[:, 0]).size)

No of emails in training file: 4014


In [8]:
print('No of emails in testing file:', np.unique(sparse_test_data[:, 0]).size)

No of emails in testing file: 1725


## Creating  an empty dataframe 

In [9]:
column_names = ['DOC_ID'] + ['CATEGORY'] + list(range(0, VOCAB_SIZE))
column_names[:5]

['DOC_ID', 'CATEGORY', 0, 1, 2]

In [10]:
len(column_names)

2502

In [11]:
index_names = np.unique(sparse_train_data[:, 0])
index_names

array([   0,    1,    2, ..., 5796, 5797, 5798])

In [12]:
full_train_data = pd.DataFrame(index=index_names, columns=column_names)
full_train_data.fillna(value=0, inplace=True)

In [13]:
full_train_data.head()

,DOC_ID,CATEGORY,0,1,2,3,4,5,6,7,...,2490,2491,2492,2493,2494,2495,2496,2497,2498,2499
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


## Create a Full Matrix From Sparse Matrix

In [65]:
def make_full_matrix(sparse_matrix, no_words, doc_idx=0, word_idx=1, cat_idx=2, freq_idx=3):
    
    """
    Form a full matrix from a sparse matrix. Return a pandas dataframe.
    Keyword arguments:
    sparse_matrix -- numpyr array
    no_words -- size of the vocabulary. Total no of tokens.
    doc_idx -- position of the document id in the sparse matrix. Default: 1st column
    word_idx -- position of the word id in the sparse matrix. Default: 2nd column
    cat_idx -- position of the label(spam is 1, nonspam is 0). Default: 3rd column
    freq_idx -- position of occurence of word in sparse matrix. Default: 4th column
    
    """
    
    column_names = ['DOC_ID'] + ['CATEGORY'] + list(range(0, VOCAB_SIZE))
    doc_id_names = np.unique(sparse_matrix[:, 0])
    full_matrix = pd.DataFrame(index=doc_id_names, columns=column_names)
    full_matrix.fillna(value=0, inplace=True)
    
    for i in range(0, sparse_matrix.shape[0]):
        doc_no = sparse_matrix[i][doc_idx]
        word_id = sparse_matrix[i][word_idx]
        label = sparse_matrix[i][cat_idx]
        occurence = sparse_matrix[i][freq_idx]
        
        full_matrix.at[doc_no, 'DOC_ID'] = doc_no
        full_matrix.at[doc_no, 'CATEGORY'] = label
        full_matrix.at[doc_no, word_id] = occurence
    
    full_matrix.set_index('DOC_ID', inplace=True)
    return full_matrix

In [67]:
%%time 
full_training_data = make_full_matrix(sparse_train_data, VOCAB_SIZE)

Wall time: 9.69 s


In [16]:
full_training_data.head()

,CATEGORY,0,1,2,3,4,5,6,7,8,...,2490,2491,2492,2493,2494,2495,2496,2497,2498,2499
DOC_ID,,,,,,,,,,,,,,,,,,,,,
0,1,0,0,1,2,1,0,0,3,0,...,0,0,0,0,0,0,0,0,0,0
1,1,7,1,2,0,1,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
2,1,6,1,1,0,1,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
3,1,6,0,0,2,4,0,3,13,0,...,0,0,0,0,0,0,0,0,0,0
4,1,5,1,2,0,1,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0


# Training the Naive Bayes Model

## Calculating the probablity of Spam

In [17]:
full_training_data.CATEGORY.size

4014

In [18]:
full_training_data.CATEGORY.sum()

1248

In [19]:
spam_prob = full_training_data.CATEGORY.sum() / full_training_data.CATEGORY.size
print('Probability of spam is', spam_prob)

Probability of spam is 0.3109118086696562


## Total Number Of Words / Tokens

In [20]:
full_train_features = full_training_data.loc[:, full_training_data.columns != 'CATEGORY']
full_train_features.head()

,0,1,2,3,4,5,6,7,8,9,...,2490,2491,2492,2493,2494,2495,2496,2497,2498,2499
DOC_ID,,,,,,,,,,,,,,,,,,,,,
0,0,0,1,2,1,0,0,3,0,0,...,0,0,0,0,0,0,0,0,0,0
1,7,1,2,0,1,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
2,6,1,1,0,1,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
3,6,0,0,2,4,0,3,13,0,0,...,0,0,0,0,0,0,0,0,0,0
4,5,1,2,0,1,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0


In [21]:
email_lengths = full_train_features.sum(axis=1) # Total number of tokens per email
email_lengths.shape

(4014,)

In [22]:
email_lengths[:5]

DOC_ID
0     87
1     53
2     40
3    182
4     43
dtype: int64

In [23]:
total_wc = email_lengths.sum()
total_wc

448976

## Number of Tokens In Spam And Ham Emails

In [24]:
spam_lengths = email_lengths[full_training_data.CATEGORY == 1]
spam_lengths.shape

(1248,)

In [25]:
spam_wc = spam_lengths.sum()
spam_wc

194865

In [26]:
ham_lengths = email_lengths[full_training_data.CATEGORY == 0]
ham_lengths.shape

(2766,)

In [27]:
nonspam_wc = ham_lengths.sum()
nonspam_wc

254111

In [28]:
email_lengths.shape[0] - spam_lengths.shape[0] - ham_lengths.shape[0]

0

In [29]:
spam_wc + nonspam_wc - total_wc

0

In [30]:
print('Average number of words in spam emails {:.0f}'.format(spam_wc / spam_lengths.shape[0]))
print('Average number of words in spam emails {:.0f}'.format(nonspam_wc / ham_lengths.shape[0]))

Average number of words in spam emails 156
Average number of words in spam emails 92


## Summing the Tokens Occuring in the Spam

In [31]:
full_train_features.shape

(4014, 2500)

In [33]:
train_spam_tokens = full_train_features.loc[full_training_data.CATEGORY == 1]
train_spam_tokens.head()

,0,1,2,3,4,5,6,7,8,9,...,2490,2491,2492,2493,2494,2495,2496,2497,2498,2499
DOC_ID,,,,,,,,,,,,,,,,,,,,,
0,0,0,1,2,1,0,0,3,0,0,...,0,0,0,0,0,0,0,0,0,0
1,7,1,2,0,1,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
2,6,1,1,0,1,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
3,6,0,0,2,4,0,3,13,0,0,...,0,0,0,0,0,0,0,0,0,0
4,5,1,2,0,1,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0


In [34]:
train_spam_tokens.tail()

,0,1,2,3,4,5,6,7,8,9,...,2490,2491,2492,2493,2494,2495,2496,2497,2498,2499
DOC_ID,,,,,,,,,,,,,,,,,,,,,
1887,0,0,0,0,0,0,0,2,2,0,...,0,0,0,0,0,0,0,0,0,0
1889,0,0,2,3,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1890,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,1,0,0,0,0
1895,0,3,1,1,1,1,44,1,5,5,...,2,0,4,0,0,0,0,0,0,0
1896,1,1,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [35]:
train_spam_tokens.shape

(1248, 2500)

In [39]:
summed_spam_tokens = train_spam_tokens.sum(axis=0) + 1
summed_spam_tokens.shape

(2500,)

In [40]:
summed_spam_tokens.tail()

2495    21
2496     7
2497     1
2498     3
2499    30
dtype: int64

## Summing the Tokens Occuring in Ham

In [43]:
train_ham_tokens = full_train_features.loc[full_training_data.CATEGORY == 0]
train_ham_tokens.head()

,0,1,2,3,4,5,6,7,8,9,...,2490,2491,2492,2493,2494,2495,2496,2497,2498,2499
DOC_ID,,,,,,,,,,,,,,,,,,,,,
1898,1,2,1,0,1,1,2,0,1,1,...,0,0,0,0,0,0,0,0,0,0
1899,2,1,0,1,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
1900,2,1,0,1,1,0,2,1,0,0,...,0,0,0,0,0,0,0,0,0,0
1901,2,1,1,0,0,1,1,0,1,0,...,0,0,0,0,0,0,0,0,0,0
1903,2,2,0,1,0,0,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0


In [44]:
train_ham_tokens.tail()

,0,1,2,3,4,5,6,7,8,9,...,2490,2491,2492,2493,2494,2495,2496,2497,2498,2499
DOC_ID,,,,,,,,,,,,,,,,,,,,,
5792,3,1,0,1,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
5795,2,2,0,1,0,0,2,1,0,0,...,0,0,0,0,0,0,0,0,0,0
5796,1,9,1,0,0,1,2,1,1,1,...,0,0,0,0,0,0,0,0,0,0
5797,1,1,1,0,0,1,2,0,0,1,...,0,0,0,0,0,0,0,0,0,0
5798,3,4,2,0,5,0,3,0,0,4,...,0,0,0,0,0,0,0,0,0,0


In [45]:
train_ham_tokens.shape

(2766, 2500)

In [48]:
summed_ham_tokens = train_ham_tokens.sum(axis=0) + 1
summed_ham_tokens.shape

(2500,)

In [49]:
summed_ham_tokens.tail()

2495    16
2496    30
2497    36
2498    39
2499     5
dtype: int64

In [50]:
train_ham_tokens[2499].sum() + 1

5

## P(Token | Spam) - Probablity that a Token Occurs given the email                                        is spam

In [51]:
prob_tokens_spam = summed_spam_tokens / (spam_wc + VOCAB_SIZE)
prob_tokens_spam[:5]

0    0.011218
1    0.005122
2    0.006936
3    0.010402
4    0.006957
dtype: float64

In [52]:
prob_tokens_spam.sum()

1.0

## P(Token | Spam) - Probablity that a Token Occurs given the email                                        is Nonspam

In [53]:
prob_tokens_nonspam = summed_ham_tokens / (nonspam_wc + VOCAB_SIZE)
prob_tokens_nonspam.sum()

1.0

# P(Token) - Probablity that a Token Occurs

In [58]:
prob_tokens_all = full_train_features.sum() / total_wc

In [59]:
prob_tokens_all.sum()

1.0

## Save The Trained Model

In [62]:
np.savetxt(TOKEN_SPAM_PROB_FILE, prob_tokens_spam)
np.savetxt(TOKEN_HAM_PROB_FILE, prob_tokens_nonspam)
np.savetxt(TOKEN_ALL_PROB_FILE, prob_tokens_all)

# Prepare Test Data

In [80]:
sparse_test_data.shape

(112511, 4)

In [68]:
%%time
full_testing_data = make_full_matrix(sparse_test_data, VOCAB_SIZE)

Wall time: 4.67 s


In [76]:
full_testing_data.head()

,CATEGORY,0,1,2,3,4,5,6,7,8,...,2490,2491,2492,2493,2494,2495,2496,2497,2498,2499
DOC_ID,,,,,,,,,,,,,,,,,,,,,
8,1,0,0,1,4,2,1,2,4,1,...,0,0,0,0,0,0,0,0,0,0
12,1,6,1,1,0,1,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
14,1,0,0,1,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
15,1,0,2,1,1,2,0,0,3,0,...,0,0,0,0,0,0,0,0,0,0
17,1,0,0,0,0,1,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0


In [81]:
X_test = full_testing_data.loc[:, full_testing_data.columns != 'CATEGORY']
y_test = full_testing_data.CATEGORY

In [ ]:
np.savetxt(TEST_TARGET_FILE, y_)